# Concatenating COSA Datasets LOL

## Imports

In [1]:
import pandas as pd

In [ ]:
#initial dataset 
#air1 = pd.read_csv('med_center_air1.csv')

In [ ]:
#additional dataset
#air2 = pd.read_csv('med_center_air.csv')

In [ ]:
#check out datatypes and nulls
#air1.info()

In [ ]:
#checkout datatypes and nulls
#air2.info()

- O3 is a float in air 2, need to make them both floats

In [ ]:
#air1.astype({'O3': 'float64'}).dtypes

In [ ]:
#air1.astype({'O3': 'float64'})

In [ ]:
#air1.info()

In [ ]:
#air1.O3 = air1.O3.astype('float64')

In [ ]:
#air1.info()

In [ ]:
#air1.info()

## Full cosa dataset

In [ ]:
import wrangle

In [ ]:
def full_daily_COSA_dataframe():
    sound_df = wrangle.wrangle_sound()
    sound_df = sound_df.set_index('DateTime')
    sound_df = sound_df.sort_index()
    flood_df = wrangle.clean_flood()
    flood_df = flood_df.set_index('datetime')
    weather_df = wrangle.wrangle_weather()
    air_df = wrangle.clean_air()
    air_df.datetime = pd.to_datetime(air_df.datetime)
    air_df = air_df.set_index('datetime')
    air_df = air_df.sort_index()
    weather_day_df = weather_df.resample('D', on='datetime').mean()
    flood_day_df = flood_df.resample('D').mean()
    sound_day_df = sound_df.resample('D').mean()
    air_day_df = air_df.resample('D').mean().drop(columns = ['hour', 'weekday', 'CO_24hr', 'Pm_25_24hr', 'Pm_10_24hr', 'SO2', 'O3', 'NO2'])
    air2_5 = air_df.drop(air_df.columns.difference(['Pm2_5', 'AQI_pm2_5']), 1)
    air10 = air_df.drop(air_df.columns.difference(['Pm10', 'AQI_pm10']), 1)
    airCO = air_df.drop(air_df.columns.difference(['CO', 'AQI_CO']), 1)
    series2_5 = air2_5.resample('D').max().rename(columns = {'AQI_pm2_5': 'most_hazardous_pm2.5_level'})['most_hazardous_pm2.5_level']
    series10 = air10.resample('D').max().rename(columns = {'AQI_pm10': 'most_hazardous_pm10_level'})['most_hazardous_pm10_level']
    seriesCO = airCO.resample('D').max().rename(columns = {'AQI_CO': 'most_hazardous_CO_level'})['most_hazardous_CO_level']
    hazards = pd.DataFrame(series2_5).join(series10).join(seriesCO)
    df = weather_day_df.join(air_day_df).join(hazards).join(sound_day_df).join(flood_day_df)
    df = df.round({'celsius': 2, 'farenheit': 2, 'humidity': 2, 'dewpoint_celsius': 2, 'dewpoint_farenheit': 2,
          'pressure': 2, 'NoiseLevel_db': 2, 'sensor_to_water_feet': 2, 'sensor_to_water_meters': 2,
          'sensor_to_ground_feet': 2, 'sensor_to_ground_meters': 2, 'flood_depth_feet': 2,
          'flood_depth_meters': 2})
    return df

In [ ]:
df = full_daily_COSA_dataframe()


In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
def all_hours_daily_COSA_dataframe():
    sound_df = wrangle.wrangle_sound()
    sound_df = sound_df.set_index('DateTime')
    sound_df = sound_df.sort_index()
    flood_df = wrangle.clean_flood()
    flood_df = flood_df.set_index('datetime')
    weather_df = wrangle.wrangle_weather()
    air_df = wrangle.clean_air()
    air_df.datetime = pd.to_datetime(air_df.datetime)
    air_df = air_df.set_index('datetime')
    air_df = air_df.sort_index()
    weather_hour_df = weather_df.resample('1H', on='datetime').mean()
    flood_hour_df = flood_df.resample('1H').mean()
    sound_hour_df = sound_df.resample('1H').mean()
    air_hour_df = air_df.resample('1H').mean().drop(columns = ['hour', 'weekday', 'CO_24hr', 'Pm_25_24hr', 'Pm_10_24hr', 'SO2', 'O3', 'NO2'])
    air2_5 = air_df.drop(air_df.columns.difference(['Pm2_5', 'AQI_pm2_5']), 1)
    air10 = air_df.drop(air_df.columns.difference(['Pm10', 'AQI_pm10']), 1)
    airCO = air_df.drop(air_df.columns.difference(['CO', 'AQI_CO']), 1)
    series2_5 = air2_5.resample('1H').max().rename(columns = {'AQI_pm2_5': 'most_hazardous_pm2.5_level'})['most_hazardous_pm2.5_level']
    series10 = air10.resample('1H').max().rename(columns = {'AQI_pm10': 'most_hazardous_pm10_level'})['most_hazardous_pm10_level']
    seriesCO = airCO.resample('1H').max().rename(columns = {'AQI_CO': 'most_hazardous_CO_level'})['most_hazardous_CO_level']
    hazards = pd.DataFrame(series2_5).join(series10).join(seriesCO)
    df = weather_hour_df.join(air_hour_df).join(hazards).join(sound_hour_df).join(flood_hour_df)
    df = df.round({'celsius': 2, 'farenheit': 2, 'humidity': 2, 'dewpoint_celsius': 2, 'dewpoint_farenheit': 2,
          'pressure': 2, 'NoiseLevel_db': 2, 'sensor_to_water_feet': 2, 'sensor_to_water_meters': 2,
          'sensor_to_ground_feet': 2, 'sensor_to_ground_meters': 2, 'flood_depth_feet': 2,
          'flood_depth_meters': 2})
    return df

In [ ]:
hour_df = all_hours_daily_COSA_dataframe()

In [ ]:
hour_df.head()

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
plt.rc('figure', figsize=(20, 7))
plt.style.use('tableau-colorblind10')
plt.rc('font', size=16)
plt.plot(hour_df.index, hour_df.farenheit)
plt.show()

In [ ]:
#hour_df.to_csv('hours.csv')

In [ ]:
hour_df.shape

In [ ]:
#monthly roll up
def all_months_daily_COSA_dataframe():
    sound_df = wrangle.wrangle_sound()
    sound_df = sound_df.set_index('DateTime')
    sound_df = sound_df.sort_index()
    flood_df = wrangle.clean_flood()
    flood_df = flood_df.set_index('datetime')
    weather_df = wrangle.wrangle_weather()
    air_df = wrangle.clean_air()
    air_df.datetime = pd.to_datetime(air_df.datetime)
    air_df = air_df.set_index('datetime')
    air_df = air_df.sort_index()
    weather_month_df = weather_df.resample('M', on='datetime').mean()
    flood_month_df = flood_df.resample('M').mean()
    sound_month_df = sound_df.resample('M').mean()
    air_month_df = air_df.resample('M').mean().drop(columns = ['hour', 'weekday', 'CO_24hr', 'Pm_25_24hr', 'Pm_10_24hr'])
    air2_5 = air_df.drop(air_df.columns.difference(['Pm2_5', 'AQI_pm2_5']), 1)
    air10 = air_df.drop(air_df.columns.difference(['Pm10', 'AQI_pm10']), 1)
    airCO = air_df.drop(air_df.columns.difference(['CO', 'AQI_CO']), 1)
    series2_5 = air2_5.resample('M').max().rename(columns = {'AQI_pm2_5': 'most_hazardous_pm2.5_level'})['most_hazardous_pm2.5_level']
    series10 = air10.resample('M').max().rename(columns = {'AQI_pm10': 'most_hazardous_pm10_level'})['most_hazardous_pm10_level']
    seriesCO = airCO.resample('M').max().rename(columns = {'AQI_CO': 'most_hazardous_CO_level'})['most_hazardous_CO_level']
    hazards = pd.DataFrame(series2_5).join(series10).join(seriesCO)
    df = weather_month_df.join(air_month_df).join(hazards).join(sound_month_df).join(flood_month_df)
    df = df.round({'celsius': 2, 'farenheit': 2, 'humidity': 2, 'dewpoint_celsius': 2, 'dewpoint_farenheit': 2,
          'pressure': 2, 'NoiseLevel_db': 2, 'sensor_to_water_feet': 2, 'sensor_to_water_meters': 2,
          'sensor_to_ground_feet': 2, 'sensor_to_ground_meters': 2, 'flood_depth_feet': 2,
          'flood_depth_meters': 2})
    return df

In [ ]:
months = all_months_daily_COSA_dataframe()

In [ ]:
months.head()

In [ ]:
months.info()

In [ ]:
weather = wrangle.wrangle_weather()

In [ ]:
weather.head()

In [ ]:
weather.info()

In [ ]:
saws = wrangle.wrangle_saws()

In [ ]:
saws.head()

In [ ]:
saws.isna().sum()

In [ ]:
#csv for exploration in tableauy
#saws.to_csv('clean_saws.csv')

In [ ]:
# Wrangle SAWS
def wrangle_saws():
    '''This function will drop unnecessary columns, 
    create a 'location' using data acquired from 
    other columns, and melt the data to make month year column'''
    # Reads the csv
    saws = pd.read_csv('med_center_saws.csv')
    # Removes NaN values from 'Prefix' and 'Suffix' column for concatenation in 'location'
    saws['Prefix'] = saws.Prefix.fillna(value = '')
    saws['Suffix'] = saws.Suffix.fillna(value = '')
    # Concatenating columns together for specific location
    saws['location'] = saws['Prefix'] + ' ' + saws['Service Location'] + ' ' + saws['Suffix']
    # Stripping any extra whitespace
    saws['location'] = saws.location.str.strip()
    saws = saws.drop(columns=['Unnamed: 0', 'Prefix', 'Suffix', 'Service Location'])
    saws = saws.melt(id_vars=['Record #', 'ZIP Code', 'location'], 
              var_name='Month & Year', value_name='Gallons Consumed')
    saws = saws.set_index('Record #')
    saws = saws.fillna(0)
    saws = saws.rename(columns={"ZIP Code": "zipcode", 'Month & Year':'year_month', 
                                'Gallons Consumed':'gallons_consumed'})
    # replace * with 0
    saws = saws.replace(to_replace='*', value=0)
    # change data type
    saws['gallons_consumed'] = saws['gallons_consumed'].astype(int)
    return saws
    

In [ ]:
saws.year_month

In [ ]:
try:
    pd.to_datetime(saws.year_month)
except ValueError as e:
    print('ValueError', e)

In [ ]:
saws.info()

In [ ]:
saws['year_month'] = '01-20' + saws['year_month']

In [ ]:
saws.head()

In [ ]:
saws.year_month = pd.to_datetime(saws.year_month)


In [ ]:
saws.head()

In [ ]:
len(saws.year_month.value_counts())

In [ ]:
saws.info()

In [ ]:
def fix_dates(saws):
    '''
    Function to fix year month column into
    datetime.  Adds arbitraty day. but keeps 
    the same month and year.  
    '''
    saws['datetime'] = '01-20' + saws['year_month']
    saws.datetime = pd.to_datetime(saws.datetime)
    return saws

In [ ]:
saws = fix_dates(saws)

In [ ]:
saws.info()

In [ ]:
saws.head()

In [2]:
def fix_dates(saws):
    '''
    Function to fix year month column into
    datetime.  Adds arbitraty day. but keeps 
    the same month and year.  
    '''
    saws['datetime'] = '01-20' + saws['year_month']
    saws.datetime = pd.to_datetime(saws.datetime)
    return saws

def wrangle_saws():
    '''This function will drop unnecessary columns, 
    create a 'location' using data acquired from 
    other columns, and melt the data to make month year column'''
    # Reads the csv
    saws = pd.read_csv('med_center_saws.csv')
    # Removes NaN values from 'Prefix' and 'Suffix' column for concatenation in 'location'
    saws['Prefix'] = saws.Prefix.fillna(value = '')
    saws['Suffix'] = saws.Suffix.fillna(value = '')
    # Concatenating columns together for specific location
    saws['location'] = saws['Prefix'] + ' ' + saws['Service Location'] + ' ' + saws['Suffix']
    # Stripping any extra whitespace
    saws['location'] = saws.location.str.strip()
    saws = saws.drop(columns=['Unnamed: 0', 'Prefix', 'Suffix', 'Service Location'])
    saws = saws.melt(id_vars=['Record #', 'ZIP Code', 'location'], 
              var_name='Month & Year', value_name='Gallons Consumed')
    saws = saws.set_index('Record #')
    saws = saws.fillna(0)
    saws = saws.rename(columns={"ZIP Code": "zipcode", 'Month & Year':'year_month', 
                                'Gallons Consumed':'gallons_consumed'})
    # replace * with 0
    saws = saws.replace(to_replace='*', value=0)
    # change data type
    saws['gallons_consumed'] = saws['gallons_consumed'].astype(int)

    return fix_dates(saws)

In [3]:
saws = wrangle_saws()

In [4]:
saws.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 145776 entries, 274262 to 545921
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   zipcode           145776 non-null  int64         
 1   location          145776 non-null  object        
 2   year_month        145776 non-null  object        
 3   gallons_consumed  145776 non-null  int64         
 4   datetime          145776 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 6.7+ MB


In [5]:
saws.head()

,zipcode,location,year_month,gallons_consumed,datetime
Record #,,,,,
274262,78229,ACCOLON DR,17-JAN,6733,2017-01-01
274263,78229,ACCOLON DR,17-JAN,5237,2017-01-01
274264,78229,ACCOLON DR,17-JAN,2992,2017-01-01
274265,78229,ACCOLON DR,17-JAN,0,2017-01-01
274266,78229,ACCOLON DR,17-JAN,4489,2017-01-01


In [ ]:
saws.isna().sum()